In [1]:
from simulator import *

seeds = [0, 101, 202, 303, 404, 505, 606, 707, 808, 909]

# Simple Maze Environment


In [2]:
simple_maze = """
||||||||||||||||||||||
|....................|
|..|||||.....|..|....|
|..|   |...||||||||..|
|..||| |........|....|
|.....| |||||||.|||..|
|.....|       |.|....|
|.....|       |......|
|||||||  ||||||......|
         |.......|...|
         |.......|||.|
         |...........|
         |||||||||||||
"""


def simple_env_generator(seed):
    return create_env(make_map(simple_maze, 5, premapped=False, start=(3, 6), stairs=(10, 10), seed=seed),
                      apple_reward=0.75,
                      penalty_time=-0.01)


simple_env = simple_env_generator(0)
simple_env.reset()
simple_env.render()


Hello Agent, welcome to NetHack!  You are a neutral human Caveman.              
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                            --                                                  
                            |.                                                  
                            |..                                                 
                            |..                                                 
                            |..----                                             
                            |%....|                                             
                           

# Complex Maze Environment

In [3]:
complex_maze = """
               |||||||||||||||||
 ||||||||||    |...............|
 |........|    |............|..|
 |........|    |............|..|
 ||||||...|    |..|||||..||||..|
      |...|    |..|   |........|
      |...|    |..|   |........|  ||||||
|||||||...||||||..|   |.....||||  |....|
|.................|   |.....|     |....|
|...||||....|..|..|   |.....|||||||....|
|....| |..|||||||.|   |................|
|....| |.......|..|   |................|
|....| |||||||.||.|   |....||||||||....|
|....|       |.|..|   |....|      |....|
|....|       |....|   |....||||   |....|
||||||  ||||||....|   |.......|   ||||||
        |......|..|   |.......|
        |......||.|   |.......|
        |.........|   |||||||||
        |||||||||||
"""


def complex_env_generator(seed):
    return create_env(make_map(complex_maze, 5, premapped=False, start=(3, 6), stairs=(37, 13), seed=seed),
                      apple_reward=0.75,
                      penalty_time=-0.01)


complex_env = complex_env_generator(0)
complex_env.reset()
complex_env.render()


Hello Agent, welcome to NetHack!  You are a neutral human Caveman.              
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                    -----                                                       
                    |...                                                        
                    |.%.--                                                      
                    |....| 

# Benchmarking Online Algorithms

The simulation code have been adapted to not rely on a fixed, fully observable map, but rather to generate the path dynamically based on the current state of the environment. This allows for more flexibility and adaptability in the pathfinding algorithms but comes with a performance cost due to the need to recompute the path at each step.

As the stairs won't be visible until the agent reaches the end of the maze, we will use a Frontier Search algorithm to decide the target position to explore the environment. Once again, we will use the same algorithms as in the offline benchmark to path to the chosen target, to see which one manage to find the stairs first and collect the most apples along the way.



## A* Star Online Algorithm

In [4]:
from algorithms_online import planner_online
from algorithms import a_star_apple
import numpy as np

In [5]:
param_grid = {
    'weight': np.linspace(0.2, 2, 5),
    'apple_bonus': np.linspace(0.5, 5, 5)
}

a_star_apple_df = benchmark_simulation(simple_env_generator, planner_online, seeds, param_grid,
                                       online=True, planner_func=a_star_apple)

Benchmarking: 100%|██████████| 250/250 [01:33<00:00,  2.66it/s]


In [6]:
a_star_online_avg = a_star_apple_df.groupby(['weight', 'apple_bonus']).agg(
    avg_reward=('reward', 'mean'),
    avg_path_length=('path_length', 'mean'),
    avg_apples=('apples_eaten', 'mean'),
    success_rate=('success', 'mean'),
    avg_planning_time=('planning_time', 'mean')
).reset_index()
a_star_online_avg

,weight,apple_bonus,avg_reward,avg_path_length,avg_apples,success_rate,avg_planning_time
0,0.20,0.500,2.389,61.8,2.4,1.0,0.079467
1,0.20,1.625,3.622,74.8,4.2,1.0,0.071158
2,0.20,2.750,3.671,80.4,4.3,1.0,0.069183
3,0.20,3.875,3.655,81.9,4.3,1.0,0.071129
4,0.20,5.000,3.715,86.8,4.4,1.0,0.067046
5,0.65,0.500,1.997,64.0,1.9,1.0,0.081240
6,0.65,1.625,3.298,68.9,3.7,1.0,0.066894
7,0.65,2.750,3.465,76.3,4.0,1.0,0.069387
8,0.65,3.875,3.601,79.4,4.2,1.0,0.067538
9,0.65,5.000,3.489,85.6,4.1,1.0,0.072080


In [7]:
a_star_apple_df_complex = benchmark_simulation(complex_env_generator, planner_online, seeds, param_grid,
                                               online=True, planner_func=a_star_apple)
a_star_online_complex_avg = a_star_apple_df_complex.groupby(['weight', 'apple_bonus']).agg(
    avg_reward=('reward', 'mean'),
    avg_path_length=('path_length', 'mean'),
    avg_apples=('apples_eaten', 'mean'),
    success_rate=('success', 'mean'),
    avg_planning_time=('planning_time', 'mean')
).reset_index()
a_star_online_complex_avg

Benchmarking: 100%|██████████| 250/250 [02:45<00:00,  1.51it/s]


,weight,apple_bonus,avg_reward,avg_path_length,avg_apples,success_rate,avg_planning_time
0,0.20,0.500,1.605,68.5,1.5,1.0,0.171902
1,0.20,1.625,2.327,115.6,3.0,1.0,0.187464
2,0.20,2.750,2.654,149.2,3.8,1.0,0.361194
3,0.20,3.875,2.557,160.6,3.8,1.0,0.250860
4,0.20,5.000,2.712,139.9,3.7,1.0,0.208021
5,0.65,0.500,1.521,97.3,1.7,1.0,0.176414
6,0.65,1.625,2.122,124.7,2.8,1.0,0.344016
7,0.65,2.750,2.258,123.8,3.0,1.0,0.198304
8,0.65,3.875,2.170,108.5,2.7,1.0,0.188994
9,0.65,5.000,2.154,100.9,2.6,1.0,0.160211


## Monte Carlo Tree Search Online Algorithm

In [8]:
from MCTS import mcts

param_grid = {
    'C': np.linspace(1.0, 5.0, 5),
}

mcts_df = benchmark_simulation(simple_env_generator, planner_online, seeds, param_grid,
                               online=True, planner_func=mcts)


Benchmarking: 100%|██████████| 50/50 [01:10<00:00,  1.40s/it]


In [9]:
mcts_online_avg = mcts_df.groupby(['C']).agg(
    avg_reward=('reward', 'mean'),
    avg_path_length=('path_length', 'mean'),
    avg_apples=('apples_eaten', 'mean'),
    success_rate=('success', 'mean'),
    avg_planning_time=('planning_time', 'mean')
).reset_index()
mcts_online_avg

,C,avg_reward,avg_path_length,avg_apples,success_rate,avg_planning_time
0,1.0,2.950,92.0,3.4,1.0,2.155806
1,2.0,3.281,83.2,3.8,1.0,1.075143
2,3.0,3.687,75.6,4.3,1.0,0.770324
3,4.0,2.800,67.2,3.0,1.0,0.786359
4,5.0,3.178,75.3,3.6,1.0,0.710426


In [10]:
mcts_online_complex_df = benchmark_simulation(complex_env_generator, planner_online, seeds, param_grid,
                                              online=True, planner_func=mcts)
mcts_online_complex_avg = mcts_online_complex_df.groupby(['C']).agg(
    avg_reward=('reward', 'mean'),
    avg_path_length=('path_length', 'mean'),
    avg_apples=('apples_eaten', 'mean'),
    success_rate=('success', 'mean'),
    avg_planning_time=('planning_time', 'mean')
).reset_index()
mcts_online_complex_avg

Benchmarking: 100%|██████████| 50/50 [03:04<00:00,  3.68s/it]


,C,avg_reward,avg_path_length,avg_apples,success_rate,avg_planning_time
0,1.0,1.956,131.7,2.6,1.0,3.290240
1,2.0,1.486,161.4,2.5,1.0,3.827392
2,3.0,1.577,113.6,2.0,1.0,2.002426
3,4.0,1.926,153.1,2.8,1.0,3.332728
4,5.0,1.587,147.3,2.5,1.0,3.142756


## Greedy Best First Search Online Algorithm

In [11]:
from utils import cached_bfs, manhattan_distance
from algorithms import greedy_best_first_search

param_grid = {
    'heuristic': [cached_bfs, manhattan_distance],
}

simple_greedy_df = benchmark_simulation(simple_env_generator, planner_online, seeds, param_grid,
                                        planner_func=greedy_best_first_search, online=True)

Benchmarking: 100%|██████████| 20/20 [00:09<00:00,  2.12it/s]


In [12]:
# convert the heuristic to a string for better readability
simple_greedy_df['heuristic'] = simple_greedy_df['heuristic'].apply(lambda x: x.__name__ if callable(x) else str(x))

simple_greedy_df_avg = simple_greedy_df.groupby(['heuristic']).agg(
    avg_reward=('reward', 'mean'),
    avg_path_length=('path_length', 'mean'),
    avg_apples=('apples_eaten', 'mean'),
    success_rate=('success', 'mean'),
    avg_planning_time=('planning_time', 'mean')
).reset_index()
simple_greedy_df_avg

,heuristic,avg_reward,avg_path_length,avg_apples,success_rate,avg_planning_time
0,cached_bfs,3.973,78.3,4.7,1.0,0.223289
1,manhattan_distance,4.059,81.4,4.8,1.0,0.067975


In [13]:
complex_greedy_df = benchmark_simulation(complex_env_generator, planner_online, seeds, param_grid,
                                         planner_func=greedy_best_first_search, online=True)
complex_greedy_df['heuristic'] = complex_greedy_df['heuristic'].apply(lambda x: x.__name__ if callable(x) else str(x))
complex_greedy_df_avg = complex_greedy_df.groupby(['heuristic']).agg(
    avg_reward=('reward', 'mean'),
    avg_path_length=('path_length', 'mean'),
    avg_apples=('apples_eaten', 'mean'),
    success_rate=('success', 'mean'),
    avg_planning_time=('planning_time', 'mean')
).reset_index()
complex_greedy_df_avg

Benchmarking: 100%|██████████| 20/20 [00:11<00:00,  1.78it/s]


,heuristic,avg_reward,avg_path_length,avg_apples,success_rate,avg_planning_time
0,cached_bfs,2.202,73.1,2.3,1.0,0.123010
1,manhattan_distance,2.696,132.0,3.6,1.0,0.171067


## Potential Fields Online Algorithm

In [14]:
from algorithms import potential_field_path

param_grid = {
    'weight_apple': np.linspace(.1, .3, 5),
    'modality_potential': ['max', 'sum']
}

potential_field_df = benchmark_simulation(simple_env_generator, planner_online, seeds, param_grid,
                                          online=True, planner_func=potential_field_path)

potential_field_online_avg = potential_field_df.groupby(['weight_apple', 'modality_potential']).agg(
    avg_reward=('reward', 'mean'),
    avg_path_length=('path_length', 'mean'),
    avg_apples=('apples_eaten', 'mean'),
    success_rate=('success', 'mean'),
    avg_planning_time=('planning_time', 'mean')
).reset_index()
potential_field_online_avg

Benchmarking: 100%|██████████| 100/100 [00:55<00:00,  1.80it/s]


,weight_apple,modality_potential,avg_reward,avg_path_length,avg_apples,success_rate,avg_planning_time
0,0.10,max,3.624,118.4,4.7,1.0,0.068956
1,0.10,sum,2.611,99.2,3.1,1.0,0.080118
2,0.15,max,3.507,130.7,4.7,1.0,0.069570
3,0.15,sum,2.528,101.4,3.0,1.0,0.087508
4,0.20,max,3.506,130.7,4.7,1.0,0.070065
5,0.20,sum,2.537,98.4,3.0,1.0,0.079091
6,0.25,max,3.686,120.4,4.8,1.0,0.069726
7,0.25,sum,2.640,102.9,3.2,1.0,0.077333
8,0.30,max,3.501,141.0,4.8,1.0,0.072659
9,0.30,sum,2.658,101.9,3.2,1.0,0.077481


In [15]:
potential_field_df_complex = benchmark_simulation(complex_env_generator, planner_online, seeds, param_grid,
                                                  online=True, planner_func=potential_field_path)

potential_field_online_avg_complex = potential_field_df_complex.groupby(['weight_apple', 'modality_potential']).agg(
    avg_reward=('reward', 'mean'),
    avg_path_length=('path_length', 'mean'),
    avg_apples=('apples_eaten', 'mean'),
    success_rate=('success', 'mean'),
    avg_planning_time=('planning_time', 'mean')
).reset_index()

potential_field_online_avg_complex

Benchmarking: 100%|██████████| 100/100 [01:13<00:00,  1.36it/s]


,weight_apple,modality_potential,avg_reward,avg_path_length,avg_apples,success_rate,avg_planning_time
0,0.10,max,2.092,145.7,3.1,1.0,0.112842
1,0.10,sum,0.860,126.7,1.3,1.0,0.100265
2,0.15,max,2.250,167.5,3.3,1.0,0.173106
3,0.15,sum,0.972,196.6,1.9,1.0,0.194223
4,0.20,max,2.136,146.3,3.3,1.0,0.152398
5,0.20,sum,0.820,136.4,1.3,1.0,0.129329
6,0.25,max,1.918,137.8,2.8,1.0,0.140449
7,0.25,sum,0.611,179.7,1.6,1.0,0.247660
8,0.30,max,2.271,171.2,3.6,1.0,0.170172
9,0.30,sum,0.978,104.8,1.0,1.0,0.121351


## Beam Search Online Algorithm

In [16]:
from algorithms import beam_search_apple
import numpy as np

param_grid = {
    'beam_width': np.arange(4, 11),  # from 4 to 10
    'apple_reward': np.arange(3, 11)  # from 3 to 10
}

beam_online_df = benchmark_simulation(simple_env_generator, planner_online, seeds, param_grid,
                                      online=True, planner_func=beam_search_apple)
beam_online_df_mean = beam_online_df.groupby(['beam_width', 'apple_reward']).agg(
    avg_reward=('reward', 'mean'),
    avg_path_length=('path_length', 'mean'),
    success_rate=('success', 'mean'),
    avg_apples=('apples_eaten', 'mean'),
    avg_planning_time=('planning_time', 'mean')
).reset_index()

# print the combinations with a success rate less than 1
beam_online_df_mean

Benchmarking: 100%|██████████| 560/560 [04:26<00:00,  2.10it/s]


,beam_width,apple_reward,avg_reward,avg_path_length,success_rate,avg_apples,avg_planning_time
0,4,3,3.153,70.2,1.0,3.5,0.251495
1,4,4,3.226,69.4,1.0,3.6,0.164795
2,4,5,3.292,70.4,1.0,3.7,0.171583
3,4,6,3.446,70.5,1.0,3.9,0.154972
4,4,7,3.368,69.9,1.0,3.8,0.151979
5,4,8,3.556,74.4,1.0,4.1,0.146113
6,4,9,3.470,75.1,1.0,4.0,0.145902
7,4,10,3.479,74.4,1.0,4.0,0.147027
8,5,3,3.150,70.8,1.0,3.5,0.258314
9,5,4,3.302,69.6,1.0,3.7,0.165605


In [17]:
param_grid = {
    'beam_width': np.arange(4, 9),  # from 4 to 10
    'apple_reward': np.arange(3, 7)  # from 3 to 10
}

beam_online_df_complex = benchmark_simulation(complex_env_generator, planner_online, seeds, param_grid,
                                              online=True, planner_func=beam_search_apple)

Benchmarking: 100%|██████████| 200/200 [06:07<00:00,  1.84s/it]


In [18]:
beam_online_df_complex_mean = beam_online_df_complex.groupby(['beam_width', 'apple_reward']).agg(
    avg_reward=('reward', 'mean'),
    avg_path_length=('path_length', 'mean'),
    success_rate=('success', 'mean'),
    avg_apples=('apples_eaten', 'mean'),
    avg_planning_time=('planning_time', 'mean')
).reset_index()

beam_online_df_complex_mean

,beam_width,apple_reward,avg_reward,avg_path_length,success_rate,avg_apples,avg_planning_time
0,4,3,1.758,148.4,1.0,2.6,1.877621
1,4,4,1.970,158.4,1.0,3.0,1.290207
2,4,5,1.539,111.7,1.0,1.9,0.875378
3,4,6,2.102,141.3,1.0,3.0,1.255819
4,5,3,1.597,113.4,1.0,2.0,1.280672
5,5,4,1.805,138.4,1.0,2.5,2.529411
6,5,5,1.452,89.6,1.0,1.5,1.768860
7,5,6,1.628,94.7,1.0,1.8,1.619624
8,6,3,1.243,98.1,1.0,1.4,1.016249
9,6,4,1.369,93.3,1.0,1.5,0.775036
